<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/Kaggle/2023Q2/Un_primer_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import json
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum

import pandas as pd
import matplotlib.pyplot as plt





In [2]:
df_val=pd.read_csv('dataset/testear.csv')
df_train=pd.read_csv('dataset/origen.csv')


In [3]:
# Seasons worked
for df in [df_train,df_val]:
    writers = df.groupby("directors").count()["Unnamed: 0"].to_dict()
    writers = split_and_sum(writers)
    df["directors_exp"] = df["directors"].apply(sum_into_column,args=(writers,))
    
for df in [df_val,df_train]:
    seasons = df.groupby("directors").count()["seasonNumber"].to_dict()
    seasons = split_and_sum(seasons)
    df["dir_seasonsOfExperience"] = df["directors"].apply(sum_into_column,args=(seasons,))

# episodes worked
for df in [df_val,df_train]:
    episodes = df.groupby("directors").count()["episodeNumber"].to_dict()
    episodes = split_and_sum(episodes)
    df["dir_episodesOfExperience"] = df["directors"].apply(sum_into_column,args=(episodes,))
    
# movies worked
for df in [df_val,df_train]:
    movies = df[df["titleType"] == "movie"].groupby("directors").count()["titleType"].to_dict()
    movies = split_and_sum(movies)
    df["dir_moviesOfExperience"] = df["directors"].apply(sum_into_column,args=(movies,))

# other xp than movies
for df in [df_val,df_train]:
    others = df[df["titleType"] != "movie"].groupby("directors").count()["titleType"].to_dict()
    others = split_and_sum(others)
    df["dir_othersOfExperience"] = df["directors"].apply(sum_into_column,args=(others,))

del seasons,episodes,movies,others,df

In [4]:
for df in [df_train,df_val]:
    writers = df.groupby("writers").count()["Unnamed: 0"].to_dict()
    writers = split_and_sum(writers)
    df["writers_exp"] = df["writers"].apply(sum_into_column,args=(writers,))

for df in [df_val,df_train]:
    seasons = df.groupby("writers").count()["seasonNumber"].to_dict()
    seasons = split_and_sum(seasons)
    df["writer_seasonsOfExperience"] = df["writers"].apply(sum_into_column,args=(seasons,))

for df in [df_val,df_train]:
    episodes = df.groupby("writers").count()["episodeNumber"].to_dict()
    episodes = split_and_sum(episodes)
    df["writer_episodesOfExperience"] = df["writers"].apply(sum_into_column,args=(episodes,))

for df in [df_val,df_train]:
    movies = df[df["titleType"] == "movie"].groupby("writers").count()["titleType"].to_dict()
    movies = split_and_sum(movies)
    df["writer_moviesOfExperience"] = df["writers"].apply(sum_into_column,args=(movies,))

for df in [df_val,df_train]:
    others = df[df["titleType"] != "movie"].groupby("writers").count()["titleType"].to_dict()
    others = split_and_sum(others)
    df["writer_othersOfExperience"] = df["writers"].apply(sum_into_column,args=(others,))

del seasons,episodes,movies,others,df

In [5]:
#set language to 0 if it is nan

df_train['language'] = df_train['language'].fillna(0)
df_val['language'] = df_val['language'].fillna(0)

In [6]:
y=df_train.averageRating
vars=['startYear', 'runtimeMinutes',"numVotes","writers_exp",
      "directors_exp","endYear","isAdult","seasonNumber",
      "episodeNumber","ordering","isOriginalTitle","runtime",
      "revenue","popularity","budget","dir_seasonsOfExperience",
      "dir_episodesOfExperience","dir_othersOfExperience","dir_moviesOfExperience",
      "writer_seasonsOfExperience","writer_episodesOfExperience","writer_othersOfExperience",
      "writer_moviesOfExperience"]
X=df_train[vars]
X = pd.concat([X, pd.get_dummies(df_train['language'], prefix='language')], axis=1)
X_pred=df_val[vars]
X_pred = pd.concat([X_pred, pd.get_dummies(df_val['language'], prefix='language')], axis=1)


In [7]:
for i,df in enumerate([df_train, df_val]):
    encoder = OneHotEncoder()
    title = encoder.fit_transform(df[['titleType']])
    title_df = pd.DataFrame(title.toarray(), columns=encoder.get_feature_names_out(['titleType']))
    #split by genres
    genres = df.genres_x.str.get_dummies(sep=',')
    genres = genres.reindex(sorted(genres.columns), axis=1)


    title_df = pd.concat([title_df, genres], axis=1)
    if not i:
        X = pd.concat([X, title_df], axis=1)
    else:
        X_pred = pd.concat([X_pred, title_df], axis=1)


for col in X.columns:
    if col not in X_pred.columns:
        X_pred[col] = 0

X_pred = X_pred.sort_index(axis=1)
X = X.sort_index(axis=1)
del title_df, genres, title, encoder, df





In [8]:
#drop duplicate columns
X = X.loc[:,~X.columns.duplicated()]
X_pred = X_pred.loc[:,~X_pred.columns.duplicated()]
print(X.columns)


Index(['0', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show',
       'Thriller', 'War', 'Western', 'budget', 'dir_episodesOfExperience',
       'dir_moviesOfExperience', 'dir_othersOfExperience',
       'dir_seasonsOfExperience', 'directors_exp', 'endYear', 'episodeNumber',
       'isAdult', 'isOriginalTitle', 'language_0', 'language_en',
       'language_es', 'language_fr', 'language_haw', 'language_hi',
       'language_myv', 'language_yi', 'numVotes', 'ordering', 'popularity',
       'revenue', 'runtime', 'runtimeMinutes', 'seasonNumber', 'startYear',
       'titleType_movie', 'titleType_short', 'titleType_tvEpisode',
       'titleType_tvMiniSeries', 'titleType_tvMovie', 'titleType_tvSeries',
       'titleType_tvShort', 'titleType_tvSpecial',

" ['0', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',\n       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',\n       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',\n       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show',\n       'Thriller', 'War', 'Western', 'budget', 'dir_episodesOfExperience',\n       'dir_moviesOfExperience', 'dir_othersOfExperience',\n       'dir_seasonsOfExperience', 'directors_exp', 'endYear', 'episodeNumber',\n       'isAdult', 'isOriginalTitle', 'language_en', 'language_es',\n       'language_fr', 'language_haw', 'language_hi', 'language_myv',\n       'language_yi', 'numVotes', 'ordering', 'popularity', 'revenue',\n       'runtime', 'runtimeMinutes', 'seasonNumber', 'startYear',\n       'titleType_movie', 'titleType_short', 'titleType_tvEpisode',\n       'titleType_tvMiniSeries', 'titleType_tvMovie', 'titleType_tvSeries',\n       'titleType_tvShort', 'titleType_tvSpecial', 'tit

In [9]:
X.to_csv('dataset/train.csv', index=False)
X_pred.to_csv('dataset/val.csv', index=False)